# Post imputation quality control

Here to use chr18 data from 1000 genomes.

In [1]:
using Revise
using VCFTools
using MendelImpute
using GeneticVariation
using Random
using StatsBase
using CodecZlib
using ProgressMeter
using JLSO
using BenchmarkTools
using GroupSlices
using LinearAlgebra
using UnicodePlots
# using ProfileView

BLAS.set_num_threads(1)

┌ Info: Precompiling MendelImpute [e47305d1-6a61-5370-bc5d-77554d143183]
└ @ Base loading.jl:1278


In [2]:
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")

In [12]:
# 8 threads, d = 1000
Random.seed!(2020)
chr = 18
maf = 0.1
d = 1000
overlap = 0.0
tgtfile = "target.chr$chr.typedOnly.maf$maf.masked.vcf.gz"
reffile = "ref.chr18.maxd$d.overlap$overlap.maf$maf.excludeTarget.jlso"
outfile = "mendel.imputed.vcf.gz"
@time ph, haploscore = phase(tgtfile, reffile, outfile=outfile);

X_complete = convert_gt(Float64, "target.chr18.full.vcf.gz", trans=true)
p, n = size(X_complete)
X_mendel = convert_gt(Float64, outfile, trans=true)
println("error overall = $(sum(X_mendel .!= X_complete) / n / p) \n")

Number of threads = 8
Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:00:05


Total windows = 2367, averaging ~ 527 unique haplotypes per window.

Timings: 
    Data import                     = 23.7672 seconds
        import target data             = 5.33822 seconds
        import compressed haplotypes   = 18.4289 seconds
    Computing haplotype pair        = 4.87321 seconds
        BLAS3 mul! to get M and N      = 0.363539 seconds per thread
        haplopair search               = 3.99455 seconds per thread
        initializing missing           = 0.0257177 seconds per thread
        allocating and viewing         = 0.0701098 seconds per thread
        index conversion               = 0.00365273 seconds per thread
    Phasing by win-win intersection = 1.64581 seconds
        Window-by-window intersection  = 0.10344 seconds per thread
        Breakpoint search              = 1.24049 seconds per thread
        Recording result               = 0.0483477 seconds per thread
    Imputation                     = 1.96674 seconds
        Imputing missing              

## Plot least square error for each sample

In [13]:
# normalize hapscore by number of SNPs
tot_hapscore = sum.(haploscore) ./ p
@show histogram(tot_hapscore)

# find samples with avg error ≤ 0.001
keep_idx = findall(x -> x ≤ 0.001, tot_hapscore)
npruned = length(keep_idx)
println("error pruned = $(sum(X_mendel[:, keep_idx] .!= X_complete[:, keep_idx]) / npruned / p) \n")

histogram(tot_hapscore) =                       ┌                                        ┐ 
   [0.0002, 0.0004  ) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 48   
   [0.0004, 0.0006  ) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 29                 
   [0.0006, 0.0008  ) ┤▇▇▇▇▇ 6                                   
   [0.0008, 0.001   ) ┤▇▇▇▇▇▇ 8                                  
   [0.001 , 0.0012  ) ┤▇▇▇▇ 5                                    
   [0.0012, 0.0014  ) ┤▇▇ 3                                      
   [0.0014, 0.0016  ) ┤▇ 1                                       
                      └                                        ┘ 
                                      Frequency
error pruned = 0.0049533566168525976 



In [8]:
histogram(haploscore[20])

                  ┌                                        ┐ 
   [ 0.0,  2.0  ) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 2309   
   [ 2.0,  4.0  ) ┤ 33                                       
   [ 4.0,  6.0  ) ┤ 17                                       
   [ 6.0,  8.0  ) ┤ 5                                        
   [ 8.0, 10.0  ) ┤ 0                                        
   [10.0, 12.0  ) ┤ 0                                        
   [12.0, 14.0  ) ┤ 0                                        
   [14.0, 16.0  ) ┤ 0                                        
   [16.0, 18.0  ) ┤ 1                                        
   [18.0, 20.0  ) ┤ 1                                        
   [20.0, 22.0  ) ┤ 0                                        
   [22.0, 24.0  ) ┤ 0                                        
   [24.0, 26.0  ) ┤ 1                                        
                  └                                        ┘ 
                                  Frequency

**Conclusion:** For each sample, most windows are imputed well, but a few are not (e.g. hapscore $> 20$). 

## Try calculating $r^2$

According to [minimac 3 documentation](https://genome.sph.umich.edu/wiki/Minimac3_Info_File#Rsq), for each SNP we can calculate 

$$r^2 = \frac{\frac{1}{2n} \sum_{i=1}^{2n}(D_i - \hat{p})^2}{\hat{p}(1 - \hat{p})}$$

where $\hat{p}$ is the alternative allele frequency (of your GWAS data), $D_i$ is the imputed alternate allele probability at the $i$th haplotype, and $n$ is the number of GWAS samples.

In [28]:
# import imputed haplotypes
H = convert_ht(Float64, outfile, trans=true);

In [41]:
p̂ = vec(sum(H, dims=2)) ./ n ./ 2 # mean = 2maf
r² = zeros(p)
for j in 1:p
    rⱼ = 0.0
    pⱼ = p̂[j]
    # skip snps with 0 minor alleles
    if pⱼ == 0.0 || pⱼ == 1.0
        r²[j] = 1
        continue
    end
    for i in 1:2n
        rⱼ += (H[j, i] - pⱼ)^2
    end
    r²[j] = rⱼ / 2n / (pⱼ * (1 - pⱼ))
end

In [42]:
histogram(p̂)

                ┌                                        ┐ 
   [0.0 , 0.05) ┤▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇ 620983   
   [0.05, 0.1 ) ┤▇▇ 48182                                  
   [0.1 , 0.15) ┤▇ 28152                                   
   [0.15, 0.2 ) ┤▇ 20780                                   
   [0.2 , 0.25) ┤▇ 17265                                   
   [0.25, 0.3 ) ┤▇ 14750                                   
   [0.3 , 0.35) ┤▇ 12579                                   
   [0.35, 0.4 ) ┤▇ 10088                                   
   [0.4 , 0.45) ┤▇ 9874                                    
   [0.45, 0.5 ) ┤ 8946                                     
   [0.5 , 0.55) ┤ 8385                                     
   [0.55, 0.6 ) ┤ 6969                                     
   [0.6 , 0.65) ┤ 6224                                     
   [0.65, 0.7 ) ┤ 5873                                     
   [0.7 , 0.75) ┤ 5434                                     
   [0.75, 0.8 ) ┤ 4763                  

In [44]:
histogram(r²)

                                            ┌                                        ┐ 
   [0.999999999999988 , 0.999999999999989 ) ┤ 4                                        
   [0.999999999999989 , 0.99999999999999  ) ┤ 18                                       
   [0.99999999999999  , 0.999999999999991 ) ┤ 29                                       
   [0.999999999999991 , 0.999999999999992 ) ┤ 101                                      
   [0.999999999999992 , 0.999999999999993 ) ┤ 670                                      
   [0.999999999999993 , 0.999999999999994 ) ┤ 1035                                     
   [0.999999999999994 , 0.999999999999995 ) ┤ 4320                                     
   [0.999999999999995 , 0.999999999999996 ) ┤▇ 9532                                    
   [0.999999999999996 , 0.999999999999997 ) ┤▇▇▇ 28072                                 
   [0.999999999999997 , 0.999999999999998 ) ┤▇▇▇▇▇ 56880                               
   [0.999999999999998 , 0.999999